###Dans le cas ou on veut utiliser un seul jeux de donnees qui dure quelque minutes ou on alternent des repos/frissons , le programme permet de le labelise en lui précisant les periodes ou on est en repos et les periodes ou on simule des frissons



In [ ]:
import pandas as pd
from google.colab import files
import io

# 1️⃣ Upload du fichier
uploaded = files.upload()

# 2️⃣ Lecture
for filename in uploaded:
    df = pd.read_csv(io.StringIO(uploaded[filename].decode('latin1')))
    fichier = filename
    break

# 3️⃣ Création d'une version temporaire en secondes (pas ajoutée au DataFrame final)
time_s = (df["timestamp"] - df["timestamp"].iloc[0]) / 1000.0

# 4️⃣ 👉 Modifier ici les intervalles
intervals = [
    (0, 30, "repos"),
    (30, 60, "frisson"),
    (60, 90, "repos"),
    (90, 120, "frisson"),
    (120, 150, "repos"),
    (150, 180, "frisson"),
    (180, 210, "repos"),
    (210, 240, "frisson"),
    (240, 270, "repos"),
    (270, 300, "frisson"),
]

# 5️⃣ Fonction d’annotation
def assign_label(t):
    for start, end, label in intervals:
        if start <= t < end:
            return label
    return "inconnu"

df["label"] = time_s.apply(assign_label)

# 6️⃣ Export propre (sans time_s ajouté)
nom_sortie = fichier.replace(".csv", "_annote.csv")
df.to_csv(nom_sortie, index=False)
print(f"✅ Fichier annoté enregistré sous : {nom_sortie}")
files.download(nom_sortie)


###Découpe automatique du csv pour donner en output : un csv avec juste label repos et un autre csv juste label frisson

In [ ]:
import pandas as pd
from google.colab import files
import io

# 📥 1. Sélection du fichier
uploaded = files.upload()

# 📄 2. Chargement
for filename in uploaded:
    df = pd.read_csv(io.StringIO(uploaded[filename].decode('latin1')))
    print(f"✅ Fichier chargé : {filename} ➜ {df.shape[0]} lignes")
    break  # On ne traite qu'un seul fichier

# 🔍 3. Vérification de la colonne 'label'
if "label" in df.columns:
    for classe in ["repos", "frisson"]:
        df_filtre = df[df["label"] == classe]
        if not df_filtre.empty:
            # 🔄 Création du nom de sortie
            nom_base = filename.replace(".csv", "")
            nom_sortie = f"{nom_base}_{classe}.csv"

            # 💾 Export CSV
            df_filtre.to_csv(nom_sortie, index=False)
            files.download(nom_sortie)
            print(f"✅ Exporté : {nom_sortie} ({len(df_filtre)} lignes)")
else:
    print("❌ Ce fichier ne contient pas de colonne 'label'.")


#  Évaluation Automatique des Paramètres

Ce notebook explore automatiquement différentes combinaisons de paramètres (fenêtrage, filtre, FFT) pour classifier des mouvements `repos` vs `frisson`. Il effectue un benchmark multi-modèles et sauvegarde les résultats dans un CSV.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import warnings
warnings.filterwarnings("ignore")

##  Paramètres de recherche

In [2]:
cutoffs = [5, 10, 20]               # Fréquence de coupure
window_sizes = [50, 100, 150]       # Taille de fenêtre (échantillons)
include_freqs = [True, False]       # Avec ou sans FFT
stride_ratio = 0.5                  # Recouvrement de 50%

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver="liblinear"),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "SVM": SVC(kernel="linear")
}

metrics = {
    "accuracy": accuracy_score,
    "f1_macro": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro"),
    "precision_macro": lambda y_true, y_pred: precision_score(y_true, y_pred, average="macro"),
    "recall_macro": lambda y_true, y_pred: recall_score(y_true, y_pred, average="macro")
}

##  Chargement des fichiers CSV

In [3]:
from google.colab import files
uploaded = files.upload()

fichiers = []
for fichier in uploaded:
    label = "frisson" if "frisson" in fichier.lower() else "repos"
    fichiers.append((fichier, label))


Saving repos_rotation.csv to repos_rotation.csv
Saving repos_initial_FV.csv to repos_initial_FV.csv
Saving repos_assis_FV.csv to repos_assis_FV.csv
Saving repos_allongé_FV.csv to repos_allongé_FV.csv
Saving repos.csv to repos.csv
Saving frisson_assis_bras_table_FV.csv to frisson_assis_bras_table_FV.csv
Saving frisson_assis_bras_cote_FV.csv to frisson_assis_bras_cote_FV.csv
Saving frisson_allonge_FV.csv to frisson_allonge_FV.csv
Saving frisson.csv to frisson.csv


##  Fonctions d'extraction et filtrage

In [4]:
from scipy.signal import butter, filtfilt

def apply_lowpass_filter(df, cutoff, fs=71.4, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    df_filtered = df.copy()
    for axis in ["accel.x", "accel.y", "accel.z", "gyro.x", "gyro.y", "gyro.z"]:
        if axis in df.columns:
            df_filtered[axis] = filtfilt(b, a, df[axis])
    return df_filtered
def extract_features(segment, label, with_freqs=True, fs=50):
    features = {}
    for axe in ["accel.x", "accel.y", "accel.z", "gyro.x", "gyro.y", "gyro.z"]:
        sig = segment[axe].values
        features[f"{axe}_mean"] = np.mean(sig)
        features[f"{axe}_std"] = np.std(sig)
        features[f"{axe}_min"] = np.min(sig)
        features[f"{axe}_max"] = np.max(sig)
        features[f"{axe}_energy"] = np.sum(sig ** 2)

        if with_freqs:
            yf = fft(sig)
            xf = fftfreq(len(sig), 1 / fs)
            freqs = xf[:len(sig)//2]
            mags = np.abs(yf[:len(sig)//2])
            dom_freq = freqs[np.argmax(mags)]
            features[f"{axe}_dom_freq"] = dom_freq

    features["label"] = label
    return features

##  Boucle d'évaluation

In [5]:
results = []

for cutoff in cutoffs:
    for window in window_sizes:
        stride = int(window * stride_ratio)
        for with_freqs in include_freqs:
            features_list = []

            for nom, label in fichiers:
                df = pd.read_csv(nom, encoding="latin1")
                if "label" in df.columns:
                    df = df.drop(columns=["label"])

                df = apply_lowpass_filter(df, cutoff)

                for i in range(0, len(df) - window + 1, stride):
                    segment = df.iloc[i:i + window]
                    feats = extract_features(segment, label, with_freqs)
                    features_list.append(feats)

            df_feat = pd.DataFrame(features_list)

            X = df_feat.drop(columns=["label"])
            y = df_feat["label"]

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            for model_name, model in models.items():
                skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                for metric_name, metric_func in metrics.items():
                    scores = []
                    for train_idx, test_idx in skf.split(X_scaled, y):
                        model.fit(X_scaled[train_idx], y.iloc[train_idx])
                        y_pred = model.predict(X_scaled[test_idx])
                        score = metric_func(y.iloc[test_idx], y_pred)
                        scores.append(score)

                    results.append({
                        "cutoff": cutoff,
                        "window": window,
                        "with_freqs": with_freqs,
                        "stride": stride,
                        "model": model_name,
                        "metric": metric_name,
                        "mean_score": np.mean(scores),
                        "std_score": np.std(scores)
                    })

In [6]:
results = []

for cutoff in cutoffs:
    for window in window_sizes:
        stride = int(window * stride_ratio)
        for with_freqs in include_freqs:
            features_list = []

            for nom, label in fichiers:
                df = pd.read_csv(nom, encoding="latin1")
                if "label" in df.columns:
                    df = df.drop(columns=["label"])

                df = apply_lowpass_filter(df, cutoff)

                for i in range(0, len(df) - window + 1, stride):
                    segment = df.iloc[i:i + window]
                    feats = extract_features(segment, label, with_freqs)
                    features_list.append(feats)

            df_feat = pd.DataFrame(features_list)
            print(f"Config = cutoff:{cutoff}, window:{window}, with_freqs:{with_freqs}, stride:{stride}")
            print("Taille du dataset :", len(df_feat))
            print("Répartition :")
            print(df_feat["label"].value_counts())
            print("-"*40)

            X = df_feat.drop(columns=["label"])
            y = df_feat["label"]

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            for model_name, model in models.items():
                skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                for metric_name, metric_func in metrics.items():
                    scores = []
                    for train_idx, test_idx in skf.split(X_scaled, y):
                        model.fit(X_scaled[train_idx], y.iloc[train_idx])
                        y_pred = model.predict(X_scaled[test_idx])
                        score = metric_func(y.iloc[test_idx], y_pred)
                        scores.append(score)

                    results.append({
                        "cutoff": cutoff,
                        "window": window,
                        "with_freqs": with_freqs,
                        "stride": stride,
                        "model": model_name,
                        "metric": metric_name,
                        "mean_score": np.mean(scores),
                        "std_score": np.std(scores)
                    })

Config = cutoff:5, window:50, with_freqs:True, stride:25
Taille du dataset : 1254
Répartition :
label
repos      712
frisson    542
Name: count, dtype: int64
----------------------------------------
Config = cutoff:5, window:50, with_freqs:False, stride:25
Taille du dataset : 1254
Répartition :
label
repos      712
frisson    542
Name: count, dtype: int64
----------------------------------------
Config = cutoff:5, window:100, with_freqs:True, stride:50
Taille du dataset : 621
Répartition :
label
repos      353
frisson    268
Name: count, dtype: int64
----------------------------------------
Config = cutoff:5, window:100, with_freqs:False, stride:50
Taille du dataset : 621
Répartition :
label
repos      353
frisson    268
Name: count, dtype: int64
----------------------------------------
Config = cutoff:5, window:150, with_freqs:True, stride:75
Taille du dataset : 408
Répartition :
label
repos      232
frisson    176
Name: count, dtype: int64
----------------------------------------
Con

##  Résultats et export

In [7]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="mean_score", ascending=False)
df_results.head(20)

,cutoff,window,with_freqs,stride,model,metric,mean_score,std_score
202,20,150,True,75,SVM,precision_macro,0.997872,0.004255
200,20,150,True,75,SVM,accuracy,0.997531,0.004938
201,20,150,True,75,SVM,f1_macro,0.997475,0.005049
203,20,150,True,75,SVM,recall_macro,0.997143,0.005714
126,10,150,True,75,Random Forest,precision_macro,0.995789,0.005158
199,20,150,True,75,Random Forest,recall_macro,0.994286,0.006999
130,10,150,True,75,SVM,precision_macro,0.993706,0.005140
198,20,150,True,75,Random Forest,precision_macro,0.993661,0.005176
196,20,150,True,75,Random Forest,accuracy,0.992653,0.005999
128,10,150,True,75,SVM,accuracy,0.992653,0.005999


In [8]:
#  Sauvegarde CSV
csv_filename = "resultats_grid_search.csv"
df_results.to_csv(csv_filename, index=False)
print(f"✅ Résultats sauvegardés dans : {csv_filename}")

✅ Résultats sauvegardés dans : resultats_grid_search.csv
